# Introduction
This notebook is designed to help you get familiar with combinatorial testing, throught a few simple examples. 
We use [Genie](https://github.com/rbehjati/genie), which is a java wrapper around [jenny](http://burtleburtle.net/bob/math/jenny.html),
a combinatorial test case generator, written in C.
You can add *Genie* to your own java project using the following maven dependency:
```
<dependency>
  <groupId>io.github.rbehjati</groupId>
  <artifactId>genie</artifactId>
  <version>0.1.0</version>
</dependency>
```         
## Content
We will start with an example, where we create all combiantions for simple model.

The next example shows the use of t-wise testing, which is based on the idea that the number of test cases can be drastically reduced by focusing on covering interactions of subsets of parameters (or factors), as opposed to covering all combinations.

The last example shows the use of constraints.

In [39]:
%classpath add mvn io.github.rbehjati:genie:0.1.0
%classpath add mvn junit:junit:4.12
%classpath add mvn org.assertj:assertj-core:3.8.0

## Example 1: All combinations
This simple example shows how *Genie* can be used to generate all combinations for a model with three factors, **age**, **gender**, and **citizenship**. Age has three values (representing equivalence classes), and each of gender and citizenship has two possible values. 
Therefore, we expect the generator to generate 12 combinations.

In [54]:
import io.rbehjati.genie.CombinationGenerator;
import io.rbehjati.genie.model.Combination;
import io.rbehjati.genie.model.Factor;

import java.util.Arrays;
import java.util.List; 


CombinationGenerator generator = new CombinationGenerator();

Factor age = new Factor("age", "<18", "18-33", ">33");
Factor gender = new Factor("gender", "male", "female");
Factor citizenship = new Factor("citizenship", "European", "non-European");

int t = 3;
List<Combination> combinations = generator.generateCombinations(3, Arrays.asList(age, gender, citizenship));
System.out.println("Total numebr of combinations: " + combinations.size());


Total numebr of combinations: 12


null

## Example 2: Pairwise combinations

This example shows how the use of t-wise testing can reduce the number of test cases.
In this example we want to use t-wise testing to write test cases for a tiny library that allows specifying and manipulating logical formulas and converting them to disjunctive normal form. The code for this library can be found in the resources folder: `resources/dnf`.
There are two classes in this library: Literal and DisjunctiveNormalForm. 

In [55]:
%classpath add jar ./resources/dnf-0.0.1-SNAPSHOT.jar

### Test case
Here is the test code. The class has two helper methods for convrting combinations into instances of DisjunctiveNormalForm, and calculating the expectdResults for each combination.

Test cases are generated using the combinatorial test case generator.
Here we have three factors, and use parwise testing. A total of 5 test cases are generated and executed, showing that the DisjunctiveNormalForm does not correctly convert all formulas into minimal disjunctive normal forms!

The source code in `resources/dnf` contains an improved version of DisjunctiveNormalForm (i.e., `io.rbehjati.dnf.v2.DisjunctiveNormalForm`) that can correctly handle these cases. 

In [71]:
import static org.assertj.core.api.Assertions.assertThat;
import io.rbehjati.genie.CombinationGenerator;
import io.rbehjati.genie.model.Combination;
import io.rbehjati.genie.model.Factor;
import io.rbehjati.dnf.v1.*;
import java.util.*;

public class DisjunctiveNormalFormTest{
    
    public static Factor lhs = new Factor("lhs", "a.b", "a + c");
    public static Factor rhs = new Factor("rhs", "a.b", "a + c");
    public static Factor lhsNeg = new Factor("lhsNeg", "yes", "no");

    public void testCombinations(){
        
        CombinationGenerator generator = new CombinationGenerator();
        List<Combination> combinations = generator.generateCombinations(2, Arrays.asList(lhs, rhs, lhsNeg));

        combinations.forEach(c -> {
            DisjunctiveNormalForm lhsVal = parse(c.getEquivalenceClass(lhs));
            DisjunctiveNormalForm rhsVal = parse(c.getEquivalenceClass(rhs));
            String lhsIsNeg = c.getEquivalenceClass(Helper.lhsNeg);
            if("yes".equals(lhsIsNeg)){
                lhsVal = lhsVal.negate();
            }

            DisjunctiveNormalForm implication = DisjunctiveNormalForm.or(lhsVal.negate(), rhsVal);

            try {
                assertThat(implication.toString()).isEqualTo(expectedResult(c));
            } catch(Error e){
                System.err.println(e.getMessage());
            }
        });
    }
    
    private String expectedResult(Combination combination) {
        String lhsVal = combination.getEquivalenceClass(lhs);
        String rhsVal = combination.getEquivalenceClass(rhs);
        String lhsIsNeg = combination.getEquivalenceClass(lhsNeg);

        if (lhsVal.equals(rhsVal)) {
            if ("no".equals(lhsIsNeg)) {
                return ""; // a tautology
            } else {
                return rhsVal;
            }
        }

        if ("yes".equals(lhsIsNeg)) {
            return "a + c";
        }

        if("a.b".equals(lhsVal)){
            return "!b + c";
        }
        return "!a.!c + a.b";
    }

    private DisjunctiveNormalForm parse(String dnf) {
        if ("a.b".equals(dnf)) {
            return DisjunctiveNormalForm.and(new Literal("a"), new Literal("b"));
        } else if ("a + c".equals(dnf)) {
            return DisjunctiveNormalForm.or(new Literal("a"), new Literal("c"));
        } else {
            return DisjunctiveNormalForm.or(new Literal(dnf));
        }
    }
}


com.twosigma.beaker.javash.bkr489c24e7.DisjunctiveNormalFormTest

## Literature

1. [Pairwise Testing in Real World](http://www.pairwise.org/docs/pnsqc2006/PNSQC%20140%20-%20Jacek%20Czerwonka%20-%20Pairwise%20Testing%20-%20BW.pdf)<br/>
This article explains some of the practical issues of combinatorial testing in practice, and presents possible solutions to them.
2. [Pairwise Testing: A Best Practice That Isn’t](http://testingeducation.org/BBST/testdesign/Bach_PairwisePNSQC2004.pdf)<br/> 
This article explains some of the mythes about pariwise testing, and presents, among other things, a list of reasons where pairwise testing, if misformulated, may fail.